<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EA%B8%B0%EC%82%AC_2%ED%9A%8C_%EC%9E%91%EC%97%85%ED%98%952_%EA%B8%B0%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 89.3kB/s]


In [2]:
!kaggle datasets download -d prachi13/customer-analytics

  0% 0.00/121k [00:00<?, ?B/s]
100% 121k/121k [00:00<00:00, 16.1MB/s]


In [3]:
!unzip /content/customer-analytics.zip

Archive:  /content/customer-analytics.zip
  inflating: Train.csv               


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("/content/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train = X_train.drop(columns='ID')
X_test_id = X_test['ID']
X_test = X_test.drop(columns='ID')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print()
print(X_train.head())
print()
print(X_train.describe())
print()
print(X_train.isna().sum())
print()
print(y_train.head())
print()
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

X_train['Discount_offered'] = ( X_train['Discount_offered'] - X_train['Discount_offered'].min() ) / ( X_train['Discount_offered'].max() - X_train['Discount_offered'].min() )
X_train['Weight_in_gms'] = ( X_train['Weight_in_gms'] - X_train['Weight_in_gms'].min() ) / ( X_train['Weight_in_gms'].max() - X_train['Weight_in_gms'].min() )
X_test['Discount_offered'] = ( X_test['Discount_offered'] - X_test['Discount_offered'].min() ) / ( X_test['Discount_offered'].max() - X_test['Discount_offered'].min() )
X_test['Weight_in_gms'] = ( X_test['Weight_in_gms'] - X_test['Weight_in_gms'].min() ) / ( X_test['Weight_in_gms'].max() - X_test['Weight_in_gms'].min() )
X_train_object_cols = X_train.select_dtypes('object').columns
for col in X_train_object_cols:
    X_train[col] = LE.fit_transform(X_train[col])
    X_test[col] = LE.transform(X_test[col])

print(X_train.head())
print()
print(X_train.describe())
print()

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2022)
model.fit(X_train, y_train['Reached.on.Time_Y.N'])
print("model score: ", model.score(X_train, y_train['Reached.on.Time_Y.N']))
print()
prediction = model.predict(X_test)
output = pd.DataFrame({'ID' : X_test_id, 'Reached.on.Time_Y.N' : prediction})
print(output.head())
print()
print("test data model score: ", model.score(X_test, output['Reached.on.Time_Y.N']))
output.to_csv('20220624.csv', index=False)

(8799, 10) (2200, 10) (8799, 2) (2200, 2)

      Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
8439                B             Ship                    4                1   
10848               D             Ship                    3                2   
9477                B             Ship                    4                4   
2967                B             Ship                    6                2   
6963                B             Road                    5                1   

       Cost_of_the_Product  Prior_purchases Product_importance Gender  \
8439                   284                4                low      F   
10848                  201                4             medium      M   
9477                   181                3             medium      F   
2967                   236                4                low      F   
6963                   282                4                low      F   

       Discount_offered  Weight_in_gm

In [19]:
answer = pd.read_csv('/content/20220624.csv')
print(answer.head())

      ID  Reached.on.Time_Y.N
0   8285                    0
1  10192                    0
2   8675                    0
3   5753                    0
4    448                    1
